In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from keras import models, optimizers, backend
from keras.layers import core, convolutional, pooling, InputLayer, Input
from sklearn import model_selection
import cv2
import random
from tensorflow import keras

from tensorflow.keras.utils import Sequence
import matplotlib.image as mpimg

import json
import matplotlib.pyplot as plt

from PIL import Image
import skimage 
from skimage.transform import resize

from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from sklearn.preprocessing import LabelEncoder


from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
from lime import lime_image

from keras.applications import inception_v3 as inc_net
from keras.applications.imagenet_utils import decode_predictions




In [2]:
dataset = ('C:/Users/Ketan/Thesis/self-driving-car/challenges/resized_dataset_udacity_320x240')

In [3]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'out.csv'))
# # Split data into training and validation sets
df_train, df_val = model_selection.train_test_split(df, test_size=.2)


In [6]:
pd.options.mode.chained_assignment = None #default = warn

In [7]:
df_train['angle_range'] = pd.qcut(df_train['angle'], q=10)
df_val['angle_range'] = pd.qcut(df_val['angle'], q=10)

In [8]:
df_train['labels']= df_train['angle_range'].cat.codes
df_val['labels']= df_val['angle_range'].cat.codes

In [9]:
df_train['labels'] = df_train['labels'].apply(str)
df_val['labels'] = df_val['labels'].apply(str)


In [25]:
datagen=ImageDataGenerator(rescale=1./255, horizontal_flip=True)


In [26]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="File", y_col="labels", 
                                            class_mode="categorical", target_size=(240, 320), batch_size=32)

Found 4491 validated image filenames belonging to 10 classes.


In [27]:
val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, x_col="File", y_col="labels", 
                                            class_mode="raw", target_size=(240, 320), batch_size=32)

Found 1123 validated image filenames.


In [13]:
inputs = Input(shape=(240, 320, 3))

In [14]:
from tensorflow.keras.applications import ResNet50


In [15]:
base_model = ResNet50(include_top=False, input_tensor=inputs, weights="imagenet", input_shape = (240,320,3))

In [16]:
base_model.trainable = False

In [17]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 320, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 160, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [18]:
fc1 = core.Dense(512, activation='relu', name='dense_1', kernel_regularizer= keras.regularizers.L2(l2=1e-3))
fc2 = core.Dense(256, activation='relu', name='dense_2', kernel_regularizer= keras.regularizers.L2(l2=1e-3))
fc3 = core.Dense(64, activation='relu', name='dense_3', kernel_regularizer= keras.regularizers.L2(l2=1e-3))
prediction = core.Dense(10,name='output', activation='softmax')

In [19]:
conv5_block3_out = base_model.get_layer('conv5_block3_out')
glob_avg_pooling = pooling.GlobalAveragePooling2D()
dropout = core.Dropout(.5)
x = glob_avg_pooling(conv5_block3_out.output)
x = fc1(x)
x = fc2(x)
x = fc3(x)
x = prediction(x)

model1 = tf.keras.models.Model(inputs = inputs, outputs = x)

In [20]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 320, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 160, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [21]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), loss='categorical_crossentropy', metrics=['AUC', 'categorical_accuracy'])

In [22]:
# model = models.Sequential()
# # model.add(Input(shape=(240,320))),
# model.add(base_model)
# model.add(pooling.GlobalAveragePooling2D())
# model.add(core.Dense(512, activation='relu'))
# model.add(core.Dropout(.5))
# model.add(core.Dense(256, activation='relu'))
# model.add(core.Dropout(.25))
# model.add(core.Dense(64, activation='relu'))
# model.add(core.Dense(1))
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), loss='mse')

In [28]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [29]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [30]:
model1.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks=[callback])

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
140/140 [==============================] - ETA: 0s - loss: 3.3001 - auc: 0.5056 - categorical_accuracy: 0.1074

ValueError: in user code:

    C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1323 test_function  *
        return step_function(self, iterator)
    C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1314 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Ketan\ana

In [83]:
model1.save('.h5')

In [45]:
def open_images(inference_folder: str) -> np.ndarray:
    """Loads images from a folder and prepare them for inferencing.

    Parameters
    ----------
    inference_folder : str
        Location of images for inferencing.

    Returns
    -------
    np.ndarray
        List of images as numpy arrays transformed to fit the efficient_net model input specs.
    """
    images = []
    for img in os.listdir(inference_folder):
        img_location = os.path.join(inference_folder, img)  # create full path to image

        with Image.open(img_location) as img:  # open image with pillow

            img = np.array(img)
            img = img[:, :, :3]
            img = np.expand_dims(img, axis=0)  # add 0 dimension to fit input shape of efficient_net
            img = inc_net.preprocess_input(img)

        images.append(img)
    images_array = np.vstack(images)  # combine images efficiently to a numpy array
    return images_array

In [46]:
images = open_images("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2")


In [84]:
predictions = model1.predict_generator(images)

images_names = os.listdir("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2")
for image_name, prediction in zip(images_names, predictions):
    print(image_name, prediction)

1479425574355865054.jpg [0.50977653]
1479425574405909080.jpg [0.5038983]
1479425574455953245.jpg [0.47723788]
1479425574505920417.jpg [0.43240678]
1479425574555975193.jpg [0.32270578]
1479425574605959295.jpg [0.3414632]
1479425610362131511.jpg [0.14937882]
1479425610412208898.jpg [0.24335246]
1479425610462149889.jpg [0.21629576]
1479425610512191952.jpg [0.21494599]
1479425610562195404.jpg [0.2748629]
1479425687675676802.jpg [0.0761955]
1479425687725671818.jpg [0.06019196]
1479425687775708515.jpg [0.09625836]
1479425687825720023.jpg [0.06800478]
1479425687875739298.jpg [0.07074982]
1479425720381363046.jpg [0.3067037]
1479425720431335470.jpg [0.33176264]
1479425720481412110.jpg [0.21614264]
1479425720531376518.jpg [0.18222897]
1479425720581393603.jpg [0.17789692]


In [100]:
import glob

In [106]:
def dummy(img_path):

    images = []
    img_path = "C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2"
    for filename in glob.glob('C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2/*.jpg'):
        img1 = cv2.imread(filename[2])
    return img1
    print(img1)
#         img_location = os.path.join(img_path, img)  # create full path to image
#         with cv2.imread(img_location) as img:
#             img = np.array(img)
#         images.append(img)
#     return images
            


In [107]:
dummy("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2")

In [90]:
import cv2

In [125]:


frameSize = (320, 240)
img_path = "C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2"
out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 60, frameSize)


for filename in glob.glob('img_path/*.jpg'):
    img = cv2.imread(filename)
    img.append(img)

In [129]:
img.shape[1]

320

In [95]:
img_src = cv2.imread('C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2/1479425574355865054.jpg')
print (img_src)

[[[ 76 114 138]
  [ 72 110 128]
  [ 77 111 124]
  ...
  [ 28  38  38]
  [ 37  49  49]
  [ 40  54  50]]

 [[ 39  81  93]
  [ 46  85  93]
  [ 70 105 108]
  ...
  [ 30  43  45]
  [ 31  47  46]
  [ 33  50  47]]

 [[ 65 105 104]
  [ 72 112 107]
  [ 73 108  98]
  ...
  [ 40  56  62]
  [ 32  49  52]
  [ 24  43  46]]

 ...

 [[255 253 255]
  [255 254 254]
  [254 255 253]
  ...
  [ 32  21  17]
  [ 32  21  17]
  [ 32  21  17]]

 [[254 253 255]
  [254 254 254]
  [254 254 254]
  ...
  [ 32  21  17]
  [ 32  21  17]
  [ 32  21  17]]

 [[254 254 254]
  [255 255 255]
  [255 255 255]
  ...
  [ 32  21  17]
  [ 32  21  17]
  [ 32  21  17]]]
